In [2]:
# 설치
!pip -q install konlpy mecab-python3 emoji


In [3]:

import re, emoji, collections
from konlpy.tag import Okt
tokenizer_name="okt"; okt=Okt(); mecab=None
try:
    from konlpy.tag import Mecab
    mecab=Mecab(); tokenizer_name="mecab"
except: print("[알림] Mecab 사용 불가 → Okt로 진행")

SENTS = [
    "와 ㅋㅋㅋㅋ 오늘 공연 미쳤다 🔥🔥🔥",
    "비가 와서 집콕... ㅠㅠ 커피라도 마셔야지 ☕️",
    "헐ㅋㅋ 어제 경기 ㄹㅇ 레전드;; 대박이다!!!",
    "하하하 이것도 웃기네 ㅎㅎㅎ 😂😂",
]

# 의성·의태·반복 문자 패턴 사전
PATTERNS = {
    r"(ㅋ)\1{1,}": "ㅋㅋ",   # ㅋ 반복 → ㅋㅋ
    r"(ㅎ)\1{1,}": "ㅎㅎ",   # ㅎ 반복 → ㅎㅎ
    r"(ㅠ)\1{1,}": "ㅠㅠ",   # ㅠ 반복 → ㅠㅠ
    r"(ㅜ)\1{1,}": "ㅜㅜ",   # ㅜ 반복 → ㅜㅜ
    r"(!)\1{1,}": "!",      # ! 반복 → !
    r"([?])\1{1,}": "?",    # ? 반복 → ?
}

def normalize(text):
    s = text
    for pat, rep in PATTERNS.items():
        s = re.sub(pat, rep, s)
    s = emoji.replace_emoji(s, replace=" EMOJI ")  # 이모지는 토큰 'EMOJI'로
    s = re.sub(r"\s+", " ", s).strip()
    return s

def morphs(text):
    if tokenizer_name=="mecab": return mecab.morphs(text)
    else: return okt.morphs(text, stem=True)

normed = [normalize(x) for x in SENTS]
print("정규화 결과:", normed, sep="\n")

# 토큰화 + 빈도
from collections import Counter
cnt = Counter()
for s in normed: cnt.update(morphs(s))
print("\n상위 토큰 15개:", cnt.most_common(15))


정규화 결과:
['와 ㅋㅋ 오늘 공연 미쳤다 EMOJI EMOJI EMOJI', '비가 와서 집콕... ㅠㅠ 커피라도 마셔야지 EMOJI', '헐ㅋㅋ 어제 경기 ㄹㅇ 레전드;; 대박이다!', '하하하 이것도 웃기네 ㅎㅎ EMOJI EMOJI']

상위 토큰 15개: [('EMOJI', 6), ('ㅋㅋ', 2), ('다', 2), ('와', 1), ('오늘', 1), ('공연', 1), ('미쳤', 1), ('비', 1), ('가', 1), ('와서', 1), ('집', 1), ('콕', 1), ('.', 1), ('..', 1), ('ㅠㅠ', 1)]
